In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# データ読み込み

In [ ]:
train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")

# データ分析

In [ ]:
train.head(5)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
print(train.columns)
print("-" * 10)
print(test.columns)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
df_full = pd.concat([train,test], axis=0, sort=False)
print(df_full.shape)
df_full.describe()

In [ ]:
df_full.describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9])

In [ ]:
df_full.describe(include='O')

**

In [ ]:
!pip install Jinja2==3.0.3
import pandas_profiling as pdp # pandas_ profiling の イン ポート 
display(pdp.__version__) # version の 表示 
pdp.ProfileReport(train) # レポート の 作成

# ***データを可視化***

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
#  転送の割合を確認（欠損値なし）
sns.countplot(x='Transported', data=train)
plt.title('Transported')
plt.xticks([0,1], ['非転送者','転送者'])
plt.show()

display(train['Transported'].value_counts(dropna=True))
display(train['Transported'].value_counts(dropna=True)/len(train['Transported']))

In [ ]:
#  転送の割合を確認（欠損値あり）
sns.countplot(x='Transported', data=train)
plt.title('Transported')
plt.xticks([0,1], ['非転送者','転送者'])
plt.show()

display(train['Transported'].value_counts(dropna=False))
display(train['Transported'].value_counts(dropna=False)/len(train['Transported']))

In [ ]:
# VIPか否かで転送されたかどうかを確認
sns.countplot(x='VIP', hue='Transported', data=train)
plt.title('VIPか否かで転送されたかどうか')
plt.legend(['転送されず', '転送された'])
plt.show()

# VIPとTransportedをクロス集計する
display(pd.crosstab(train['VIP'], train['Transported']))

#クロス集計しVIPごとに正規化する
display(pd.crosstab(train['VIP'], train['Transported'],normalize='index'))

> **VIPはあんまり関係ない？
> VIPじゃない人多すぎて扱うのむずそう。
> 一応、VIPだったら転送される可能性は低くなる傾向あり。**

In [ ]:
# CryoSleepか否かで転送されたかどうかを確認
sns.countplot(x='CryoSleep', hue='Transported', data=train)
plt.title('CryoSleepか否かで転送されたかどうか')
plt.legend(['非転送者', '転送者'])
plt.show()

# VIPとTransportedをクロス集計する
display(pd.crosstab(train['CryoSleep'], train['Transported']))

#クロス集計しVIPごとに正規化する
display(pd.crosstab(train['CryoSleep'], train['Transported'],normalize='index'))

> **CyroSleep状態だったら、8割がた転送されてる！
> これもCyroSleep状態の人が全体の10%強なので偏りはあるけど、使えそう！**

In [ ]:
# 年齢の分布を確認する

# 全体のヒストグラム
sns.distplot(train['Age'].dropna(), kde=False, bins=30, label='全体')

# 転送者のヒストグラム
sns.distplot(train[train['Transported']==0].Age.dropna(), kde=False, bins=30, label='非転送者')

# 非転送者のヒストグラム
sns.distplot(train[train['Transported']==1].Age.dropna(), kde=False, bins=30, label='転送者')

plt.title('乗客の年齢の分布')
plt.legend()

> **めっちゃ赤ちゃんが300人強も乗ってる！20～30代の人が多め。
> これだけだと、転送されるかと年齢の関係性は見えてこない…**

In [ ]:
# 年齢を8分割し、CategoricalAgeという変数を作成
train['CategoricalAge']=pd.cut(train['Age'],8)

# CategoricalAgeとTransportedをクロス集計する
display(pd.crosstab(train['CategoricalAge'], train['Transported']))

# クロス集計しCategoricalAgeごとに正規化する
display(pd.crosstab(train['CategoricalAge'], train['Transported'], normalize='index'))

> **乳幼児転送されがち。かわいそう。**

In [ ]:
# 目的地の種類

sns.countplot(x='Destination', data=train)
plt.title('目的地の種類')

In [ ]:
# 目的地の種類と転送の関係

sns.countplot(x='Destination',hue='Transported', data=train)
plt.legend(['非転送','転送'])
plt.title('目的地の種類と転送の関係')
plt.show()

> ***目的地だけじゃ関係性はつかめん。***

In [ ]:
# 出身の星の種類

sns.countplot(x='HomePlanet', data=train)
plt.title('出身の星の種類')

In [ ]:
# 出身の星の種類と転送の関係

sns.countplot(x='HomePlanet',hue='Transported', data=train)
plt.legend(['非転送','転送'])
plt.title('出身の星の種類と転送の関係')
plt.show()

In [ ]:
# RoomServiceの分布

sns.distplot(train['RoomService'].dropna(), kde=False, hist=True)
plt.title('RoomServiceの分布')

train['CategoricalRoomService'] = pd.qcut(train['RoomService'].rank(method='first'), 4)
train[['CategoricalRoomService', 'Transported']].groupby(['CategoricalRoomService'], as_index=False).mean()

display(pd.crosstab(train['CategoricalRoomService'], train['Transported']))

display(pd.crosstab(train['CategoricalRoomService'], train['Transported'], normalize='index'))

> **RoomSreviceを結構払った人は転送されない傾向あり！**

# 前処理

**・カテゴリ変数の変換** <p>
**HomePlanet・Destination・CryoSleep・VIP・Transportedを数値変換**

In [ ]:
# HomePlanetの変換
HomePlanets = {'Europa':0, 'Earth':1, 'Mars':2}
train['HomePlanet'] = train['HomePlanet'].map(HomePlanets)
test['HomePlanet'] = test['HomePlanet'].map(HomePlanets)

# Destinationの変換
Destinations = {'TRAPPIST-1e':0, 'PSO J318.5-22':1, '55 Cancri e':2}
train['Destination'] = train['Destination'].map(Destinations)
test['Destination'] = test['Destination'].map(Destinations)

# CryoSleepの変換
CryoSleeps = {False:0, True:1}
train['CryoSleep'] = train['CryoSleep'].map(CryoSleeps)
test['CryoSleep'] = test['CryoSleep'].map(CryoSleeps)

# VIPの変換
VIPs = {False:0, True:1}
train['VIP'] = train['VIP'].map(VIPs)
test['VIP'] = test['VIP'].map(VIPs)

# Transportedの変換
Transporteds = {False:0, True:1}
train['Transported'] = train['Transported'].map(Transporteds)

**・不要な列の削除**

In [ ]:
# PassengerId・Cabin・Nameを削除

train.drop(['PassengerId', 'Cabin', 'Name', 'CategoricalAge', 'CategoricalRoomService'], axis=1, inplace=True)
test.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)

In [ ]:
display(train.head())

# 学習

In [ ]:
X_train=train.drop(['Transported'], axis=1)
y_train=train['Transported']

**train_test_split**

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.33, random_state=0)

lgb_train = lgb.Dataset(train_x, train_y)
lgb_eval = lgb.Dataset(valid_x, valid_y)

lgbm_params = {'objective':'binary'}

# 学習
evals_result = {}
gbm = lgb.train(params = lgbm_params,
                train_set = lgb_train,
                valid_sets = [lgb_train, lgb_eval],
                early_stopping_rounds = 20,
                evals_result = evals_result,
                verbose_eval = 10);

In [ ]:
oof = (gbm.predict(valid_x) > 0.5).astype(int)
print('score:', round(accuracy_score(valid_y, oof) * 100, 2))

In [ ]:
plt.plot(evals_result['training']['binary_logloss'], label='train_loss')
plt.plot(evals_result['valid_1']['binary_logloss'], label='valid_loss')
plt.legend()

In [ ]:
test_pred = (gbm.predict(test) > 0.5).astype(int)
Transporteds2 = {0:False, 1:True}
sample_submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
sample_submission['Transported'] = test_pred
sample_submission['Transported'] = sample_submission['Transported'].map(Transporteds2)

In [ ]:
sample_submission.to_csv('train_test_split.csv', index=False)

**KFold**

In [ ]:
from sklearn.model_selection import KFold

kf3 = KFold(n_splits=3, shuffle=True)

score_list = []
models = []

for fold_,(train_index, valid_index) in enumerate(kf3.split(X_train, y_train)):
    print(f'fold{fold_+ 1}start')
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]
    
    lgb_train = lgb.Dataset(train_x, train_y)
    lgb_valid = lgb.Dataset(valid_x, valid_y)
    
    lgbm_params = {'objective':'binary'}
    
    gbm = lgb.train(params = lgbm_params,
                    train_set = lgb_train,
                    valid_sets = [lgb_train, lgb_valid],
                    early_stopping_rounds = 20,
                    verbose_eval = -1
                   )
    
    oof = (gbm.predict(valid_x) > 0.5).astype(int)
    score_list.append(round(accuracy_score(valid_y, oof) * 100, 2))
    models.append(gbm)
    print(f'fold{fold_+1}end\n')
print(score_list, '平均score:', round(np.mean(score_list), 2))

In [ ]:
models

In [ ]:
test_pred = np.zeros((len(test), 3))

for fold_, gbm in enumerate(models):
    pred_ = gbm.predict(test)
    test_pred[:, fold_] = pred_

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
print(pred)
sample_submission['Transported'] = pred
sample_submission['Transported'] = sample_submission['Transported'].map(Transporteds2)

In [ ]:
sample_submission.to_csv('3-fold_cross-validation.csv', index=False)

In [ ]:
kf5 = KFold(n_splits=5, shuffle=True)

score_list = []
models = []

for fold_,(train_index, valid_index) in enumerate(kf5.split(X_train, y_train)):
    print(f'fold{fold_+ 1}start')
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]
    
    lgb_train = lgb.Dataset(train_x, train_y)
    lgb_valid = lgb.Dataset(valid_x, valid_y)
    
    lgbm_params = {'objective':'binary'}
    
    gbm = lgb.train(params = lgbm_params,
                    train_set = lgb_train,
                    valid_sets = [lgb_train, lgb_valid],
                    early_stopping_rounds = 20,
                    verbose_eval = -1
                   )
    
    oof = (gbm.predict(valid_x) > 0.5).astype(int)
    score_list.append(round(accuracy_score(valid_y, oof) * 100, 2))
    models.append(gbm)
    print(f'fold{fold_+1}end\n')
print(score_list, '平均score:', round(np.mean(score_list), 2))

In [ ]:
test_pred = np.zeros((len(test), 5))

for fold_, gbm in enumerate(models):
    pred_ = gbm.predict(test)
    test_pred[:, fold_] = pred_

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
sample_submission['Transported'] = pred
sample_submission['Transported'] = sample_submission['Transported'].map(Transporteds2)

In [ ]:
sample_submission.to_csv('5-fold_cross-validation.csv', index=False)

In [ ]:
kf10 = KFold(n_splits=10, shuffle=True)

score_list = []
models = []

for fold_,(train_index, valid_index) in enumerate(kf10.split(X_train, y_train)):
    print(f'fold{fold_+ 1}start')
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]
    
    lgb_train = lgb.Dataset(train_x, train_y)
    lgb_valid = lgb.Dataset(valid_x, valid_y)
    
    lgbm_params = {'objective':'binary'}
    
    gbm = lgb.train(params = lgbm_params,
                    train_set = lgb_train,
                    valid_sets = [lgb_train, lgb_valid],
                    early_stopping_rounds = 20,
                    verbose_eval = -1
                   )
    
    oof = (gbm.predict(valid_x) > 0.5).astype(int)
    score_list.append(round(accuracy_score(valid_y, oof) * 100, 2))
    models.append(gbm)
    print(f'fold{fold_+1}end\n')
print(score_list, '平均score:', round(np.mean(score_list), 2))

In [ ]:
test_pred = np.zeros((len(test), 10))

for fold_, gbm in enumerate(models):
    pred_ = gbm.predict(test)
    test_pred[:, fold_] = pred_

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
sample_submission['Transported'] = pred
sample_submission['Transported'] = sample_submission['Transported'].map(Transporteds2)

In [ ]:
sample_submission.to_csv('10-fold_cross-validation.csv', index=False)